<a href="https://colab.research.google.com/github/kevinmcmanus/cas-tau/blob/master/AstropyTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Astropy

## Basics

### Constants

In [ ]:
# need the constants module
import astropy.constants as const

In [ ]:
const.G

In [ ]:
print(const.G)

In [ ]:
print (const.h)

In [ ]:
print (const.c)

See [List of Constants](https://docs.astropy.org/en/stable/constants/) for complete list

### Units

In [ ]:
import astropy.units as u

In [ ]:
u.watt

In [ ]:
632.3 * u.watt

In [ ]:
type(632.3), type(u.watt), type(632.3*u.watt)

In [ ]:
w = 632.3*u.watt

In [ ]:
type(w)

The `value` and `unit` methods on quantity objects retrieve just the value as a scalar and the unit, as in:

In [ ]:
w.value

In [ ]:
w.unit

In [ ]:
u

### Unit Conversion

In [ ]:
print(3*u.km/3*u.m)

We can convert from one set of units to another, say nanometers to Angstroms using the Quantity `to` method, below for the Hydrogen Alpha line at 656.28 nanometers:

In [ ]:
# specify the h_alpha line in nanometers
h_alpha =  656.28*u.nm

In [ ]:
#convert to angrstrom:
h_alpha.to(u.angstrom)

Divide the h_alpha wavelength: ($\lambda$) into the speed of light to get frequency:
nu = const.c/h_alpha

In [ ]:
nu = const.c/h_alpha

In [ ]:
nu

Hmmm, meters per nanometers per second, let's look at this in megahertz

In [ ]:
nu.to(u.megahertz)

In [ ]:
# plancks law, energy per photon
energy = const.h * nu.to(1/u.s)

In [ ]:
print(const.h)

In [ ]:
#want cgs?
const.h.cgs

In [ ]:
energy

In [ ]:
# apply energy each second
eps = energy/(1*u.s)

In [ ]:
eps.to(u.watt)

## Coordinates

[Coordinates Class](https://docs.astropy.org/en/stable/coordinates/)


[SkyCoords](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html)

In [ ]:
#more astropy libraries
import astropy.coordinates as coord
from astropy.coordinates.sky_coordinate import SkyCoord, EarthLocation

In [ ]:
pleiades = SkyCoord.from_name('pleiades')

In [ ]:
pleiades

In [ ]:
#get the right ascension
pleiades.ra

Notice that by default, RA comes out in degrees, minutes, seconds. To see H:M:S, convert to string with a format parmeter:

In [ ]:
pleiades.ra.to_string(u.hour)

In [ ]:
#galactic coordinates
pleiades.galactic

## Catalog Query

### Simbad

### Sloan Digital Sky Survey

## Observation Planning

## Fancy Plot

In [ ]:
!pip install astroquery

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import matplotlib.colors as colors
%matplotlib inline

In [ ]:
from astropy.coordinates import Angle
import astropy.units as u
from astroquery.simbad import Simbad
import astropy.coordinates as coord
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.table import QTable, Table, vstack
from astropy.units import Quantity

```
Byte-by-byte Description of file: nhi_hpx.dat
--------------------------------------------------------------------------------
   Bytes Format  Units   Label     Explanations
--------------------------------------------------------------------------------
   1-  8  I8     ---     HPX       HPX index (HPXINDEX)
  10- 18  F9.5   deg     RAdeg     Right ascension (J2000)
  20- 28  F9.5   deg     DEdeg     Declination (J2000)
  30- 38  F9.5   deg     GLON      Galactic longitude (J2000)
  40- 48  F9.5   deg     GLAT      Galactic latitude (J2000)
  50- 71  E22.15 cm-2    NHI       HI column density
--------------------------------------------------------------------------------
```

In [ ]:

url = 'https://cdsarc.unistra.fr/ftp/J/A+A/594/A116/nhi_hpx.dat.gz'
cols = ['HPX', 'RAdeg','DEdeg','GLON','GLAT', 'NHI']

hi4pi = pd.read_csv(url, delim_whitespace=True, names=cols, header=None)

In [ ]:
hi4pi.head()

In [ ]:
#don't run this code -- takes 20 minutes
#fig = plt.figure(figsize=(12,12))
#ax = fig.add_subplot(111)
#ax.scatter(hi4pi.RAdeg, hi4pi.DEdeg, c=hi4pi.NHI, cmap='gnuplot')
#ax.invert_xaxis()

In [ ]:
#round to 1/10 degree precision
hi4pi['RAdeg1'] = np.round(hi4pi.RAdeg,1)
hi4pi['DEdeg1'] = np.round(hi4pi.DEdeg, 1)
hi4pi['GLAT1'] = np.round(hi4pi.GLAT,1)
hi4pi['GLON1'] = np.round(hi4pi.GLON,1)

In [ ]:
#equatorial coords
hi4pi_eq = hi4pi[['RAdeg1', 'DEdeg1', 'NHI']].groupby(['RAdeg1','DEdeg1']).mean().reset_index()
hi4pi_eq.rename(columns={"RAdeg1":"ra", "DEdeg1":"dec"}, inplace=True)

#galactic coords
hi4pi_gal = hi4pi[['GLON1', 'GLAT1', 'NHI']].groupby(['GLON1','GLAT1']).mean().reset_index()
hi4pi_gal.rename(columns={"GLON1":"l", "GLAT1":"b"}, inplace=True)

In [ ]:
#convert everybody to angles and wrap the longitude
hi4pi_eq.ra = Angle(np.array(hi4pi_eq.ra)*u.degree).wrap_at(180*u.degree)
hi4pi_eq.dec = Angle(np.array(hi4pi_eq.dec)*u.degree)

hi4pi_gal.l = Angle(np.array(hi4pi_gal.l)*u.degree).wrap_at(180*u.degree)
hi4pi_gal.b = Angle(np.array(hi4pi_gal.b)*u.degree)

In [ ]:
#create the image arrays
#note declination and lattitude in 'x' (row) positions and ra and longitude in 'y' (column) positions
image_eq = pd.pivot_table(hi4pi_eq,index='dec',columns='ra', values='NHI',aggfunc=np.mean).to_numpy( copy=True)

image_gal = pd.pivot_table(hi4pi_gal,index='b',columns='l', values='NHI',aggfunc=np.mean).to_numpy( copy=True)
image_eq.shape, image_gal.shape

In [ ]:
ra_vec = Angle(np.linspace(hi4pi_eq.ra.min(), hi4pi_eq.ra.max(), image_eq.shape[1], endpoint=False)*u.degree)
dec_vec= Angle(np.linspace(hi4pi_eq.dec.min(), hi4pi_eq.dec.max(), image_eq.shape[0], endpoint=False)*u.degree)

l_vec = Angle(np.linspace(hi4pi_gal.l.min(), hi4pi_gal.l.max(), image_gal.shape[1], endpoint=False)*u.degree)
b_vec = Angle(np.linspace(hi4pi_gal.b.min(), hi4pi_gal.b.max(), image_gal.shape[0], endpoint=False)*u.degree)

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='mollweide')
ax.pcolormesh(ra_vec.radian,
           dec_vec.radian,
           image_eq,
           cmap='jet', norm=colors.LogNorm())
ax.grid()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='mollweide')
ax.pcolormesh(l_vec.radian,
           b_vec.radian,
           image_gal,
           cmap='jet', norm=colors.LogNorm())
ax.grid()

In [ ]:
clusters = ['Blanco 1',
 'Collinder 140',
 'Coma Berenices Cluster',
 'Hyades',
 'Pleiades',
 'Praesepe',
 'alpha Per']

In [ ]:
# set up the Simbad query
from astroquery.simbad import Simbad
from astropy.time import Time
mySimbad = Simbad()
mySimbad.add_votable_fields('parallax', 'pm','velocity','typed_id')
mySimbad.get_votable_fields()


In [ ]:
#from astropy.table import Table, vstack
res_table = vstack([mySimbad.query_object(c) for c in clusters],join_type='exact')

In [ ]:
def fix_table(res_table):
  cluster_info=Table(res_table['TYPED_ID', 'PLX_VALUE', 'PLX_PREC','RA', 'RA_PREC', 'DEC', 'DEC_PREC',
                              'PMRA', 'PMDEC', 'RVZ_RADVEL', 'RVZ_ERROR'])

  #gotta be a better way to do this:
  cluster_info['TYPED_ID'] = [c.decode('utf-8') for c in cluster_info['TYPED_ID']]

  #rename the columns to look like Gaia
  cluster_info.rename_column('TYPED_ID','cluster')
  cluster_info.rename_column('PLX_VALUE','parallax')
  cluster_info.rename_column('PLX_PREC', 'parallax_error')
  cluster_info.rename_column('RA', 'ra')
  cluster_info.rename_column('RA_PREC','ra_error')
  cluster_info.rename_column('DEC', 'dec')
  cluster_info.rename_column('DEC_PREC','dec_error')
  cluster_info.rename_column('PMRA', 'pmra')
  cluster_info.rename_column('PMDEC', 'pmdec')
  cluster_info.rename_column('RVZ_RADVEL','radial_velocity')
  cluster_info.rename_column('RVZ_ERROR', 'rv_error')

  #unmask the columns
  cluster_info = cluster_info.filled() 


  #index on the cluster name
  cluster_info.add_index('cluster')

  # coordinates,  note the update to J2015.5 to match with Gaia
  cluster_info['coords'] = \
      SkyCoord(ra = cluster_info['ra'],
          dec = cluster_info['dec'], unit = (u.hour, u.deg),
          obstime = 'J2000',  #simbad returns J2000 coords
          distance = coord.Distance(parallax=Quantity(cluster_info['parallax'])),
          pm_ra_cosdec = cluster_info['pmra'],
          pm_dec = cluster_info['pmdec'],
          radial_velocity = cluster_info['radial_velocity']).apply_space_motion(new_obstime=Time('2015.5',format='decimalyear'))

  return cluster_info

In [ ]:
open_cluster_info=fix_table(res_table)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='mollweide')
ax.pcolormesh(l_vec.radian,
           b_vec.radian,
           image_gal,
           cmap='viridis', norm=colors.LogNorm())

# plot the clusters
color=iter(plt.cm.rainbow(np.linspace(0,1,len(clusters))))
for c in clusters:
  ax.scatter(open_cluster_info.loc[c]['coords'].galactic.l.wrap_at(180*u.deg).radian, open_cluster_info.loc[c]['coords'].galactic.b.radian, 
    color=next(color), s=50, label=c)
ax.grid()
ax.legend(loc='upper right')

In [ ]:
glob_clusters=['NGC0104', 'NGC0288', 'NGC0362', 'NGC1851', 'NGC5272', 'NGC5904',
       'NGC6205', 'NGC6218', 'NGC6341', 'NGC6397', 'NGC6656', 'NGC6752',
       'NGC6809', 'NGC7099']

In [ ]:
#from astropy.table import Table, vstack
res_table = vstack([mySimbad.query_object(c) for c in glob_clusters],join_type='exact')

In [ ]:
glob_cluster_info = fix_table(res_table)

In [ ]:
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
fig = plt.figure(figsize=(16,12))

#gs1 = gridspec.GridSpec(1, 2, width_ratios=(1,9))

#main plot axis
#ax = plt.subplot(gs1[0,1], projection='mollweide')
ax = fig.add_subplot(111, projection='mollweide')
pcm = ax.pcolormesh(l_vec.radian,
           b_vec.radian,
           image_gal,
           cmap='Greys', norm=colors.LogNorm(),alpha=0.3)

# plot the clusters
color=iter(plt.cm.tab20(np.linspace(0,1,20)))
for c in clusters:
  ax.scatter(open_cluster_info.loc[c]['coords'].galactic.l.wrap_at(180*u.deg).radian, open_cluster_info.loc[c]['coords'].galactic.b.radian, 
    marker='P',color=next(color), s=200, label=c, alpha=1.0)
  
# plot the globs
color=iter(plt.cm.tab20(np.linspace(0,1,20)))
for c in glob_clusters:
  ax.scatter(glob_cluster_info.loc[c]['coords'].galactic.l.wrap_at(180*u.deg).radian, glob_cluster_info.loc[c]['coords'].galactic.b.radian, 
    marker='X',color=next(color), s=200, label=c, alpha=1.0)
ax.grid()
ax.legend(bbox_to_anchor=(1.25, 1), loc='upper right', borderaxespad=0.05)
#ax.legend(loc='upper right')
#ax.invert_xaxis()
ax.tick_params(axis='x', colors='white')
ax.set_xlabel('Galactic Longitude', color='White')
ax.set_ylabel('Galactic Latitude', color='white')
#put the colorbar to the left
#divider = make_axes_locatable(ax)
#cax = divider.append_axes("left", size="5%", pad=0.05)
#cax = plt.subplot(gs1[0,0])
cb = plt.colorbar(pcm, ax=[ax], location="left", shrink=0.5)
cb.ax.set_ylabel('log(NHI)')


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
axp = ax.imshow(np.random.randint(0, 100, (100, 100)))
cb = plt.colorbar(axp,ax=[ax],location='left')
plt.show()

In [ ]:
 = ax.imshow(np.arange(100).reshape((10,10)))

# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)


In [ ]:
fig = plt.figure(figsize=(12,8))

#axis for main plot
ax = fig.add_subplot(111, projection='mollweide')
pcm = ax.pcolormesh(l_vec.radian,
           b_vec.radian,
           image_gal,
           cmap='viridis', norm=colors.LogNorm(),alpha=0.3)

# plot the clusters
color=iter(plt.cm.tab20(np.linspace(0,1,20)))
for c in clusters:
  ax.scatter(open_cluster_info.loc[c]['coords'].galactic.l.wrap_at(180*u.deg).radian, open_cluster_info.loc[c]['coords'].galactic.b.radian, 
    color=next(color), s=100, label=c)
  
# plot the globs
color=iter(plt.cm.tab20(np.linspace(0,1,20)))
for c in glob_clusters:
  ax.scatter(glob_cluster_info.loc[c]['coords'].galactic.l.wrap_at(180*u.deg).radian, glob_cluster_info.loc[c]['coords'].galactic.b.radian, 
    marker='+',color=next(color), s=100, label=c)
ax.grid()
ax.legend(loc='upper right')

cb = fig.colorbar(pcm, ax=ax)
cb.ax.set_ylabel('log(NHI)',rotation=270)
#ax.invert_xaxis()

In [ ]:
# reduce resolution to 3600 x 1800
rastep = 0.1 # tenth degree steps
decstep = 0.1 # tenth degree steps


In [ ]:
hi4pi['RAint'] = (hi4pi.RAdeg//rastep).astype(int)
hi4pi['DEint'] = ((90+hi4pi.DEdeg)//decstep).astype(int)


In [ ]:
hi4pi.head()

In [ ]:
hi4pi_01 = hi4pi[['RAint','DEint','NHI']].groupby(['DEint','RAint']).mean().reset_index()
hi4pi_image = pd.pivot_table(hi4pi_01,index='DEint',columns='RAint', values='NHI',aggfunc=np.mean).to_numpy( copy=True)

In [ ]:
ra_vec = np.linspace(0,360, 3600,  endpoint=False)
dec_vec= np.linspace(-90,90,1800, endpoint=False)

In [ ]:
decdec, rara = np.meshgrid(dec_vec, ra_vec)

In [ ]:
decdec.shape

In [ ]:
xx=np.arange(10)

In [ ]:
xx[:len(xx)-1]

In [ ]:

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
ax.pcolormesh(ra_vec,dec_vec, hi4pi_image, cmap='gnuplot', norm=colors.LogNorm())
ax.invert_xaxis()

In [ ]:
from astropy.coordinates import Angle
import astropy.units as u

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='mollweide')
#ax.pcolormesh(Angle(ra_vec*u.degree).wrap_at(180*u.degree).radian,
#              Angle(dec_vec*u.degree).radian,
#              hi4pi_image, cmap='gnuplot', norm=colors.LogNorm())
ax.scatter(hi4pi1.RAdeg1, hi4pi1.DEdeg1, c=hi4pi1.NHI, cmap='gnuplot', norm=colors.LogNorm())

#ax.invert_xaxis()

In [ ]:
Angle(90*u.degree).wrap_at('180d').radian

In [ ]:
hi4pi1.head()

In [ ]:
#this takes a while too!
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='mollweide')
ax.scatter(Angle(np.array(hi4pi1.RAdeg1)*u.degree).wrap_at(180*u.degree).radian,
           Angle(np.array(hi4pi1.DEdeg1)*u.degree).radian,
           c=hi4pi1.NHI, cmap='gnuplot', norm=colors.LogNorm())
#ax.invert_xaxis()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='mollweide')
arr = np.random.rand(180, 360)

lon = np.linspace(-np.pi, np.pi,360)
lat = np.linspace(-np.pi/2., np.pi/2.,180)
Lon,Lat = np.meshgrid(lon,lat)

im = ax.pcolormesh(Lon,Lat,arr, cmap=plt.cm.jet)

In [ ]:
plt.plot(Angle(ra_vec*u.degree).radian)